In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [7]:
# TODO: FILL IN PARAMETERS
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [8]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [9]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [10]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH)

Attack dataset was already established previosly, loading dataset from "pickle/tinyimagenet/resnet34/attack_train.p".


In [11]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

### Attack Model

Simple Model

In [ ]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [ ]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [ ]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [ ]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3490, 0.7451, 0.2863,  ..., 0.6275, 0.6235, 0.6157], [0.7647, 0.7569, 0.6118,  ..., 0.6196, 0.5804, 0.5882], [0.6824, 0.5569, 0.6980,  ..., 0.6588, 0.6627, 0.6863], ..., [0.6431, 0.6588, 0.5765,  ..., 0.6863, 0.5961, 0.4078], [0.7490, 0.7255, 0.7020,  ..., 0.6000, 0.5804, 0.7961], [0.7098, 0.7333, 0.6941,  ..., 0.5686, 0.7098, 0.5843]],[[0.3490, 0.7451, 0.2863,  ..., 0.6118, 0.6078, 0.6000], [0.7647, 0.7569, 0.6118,  ..., 0.6039, 0.5647, 0.5725], [0.6824, 0.5569, 0.6980,  ..., 0.6431, 0.6353, 0.6588], ..., [0.6471, 0.6627, 0.5804,  ..., 0.6902, 0.6000, 0.4118], [0.7529, 0.7294, 0.7059,  ..., 0.6039, 0.5843, 0.8000], [0.7137, 0.7373, 0.6980,  ..., 0.5725, 0.7137, 0.5882]],[[0.3098, 0.7059, 0.2471,  ..., 0.5765, 0.5647, 0.5569], [0.7255, 0.7176, 0.5725,  ..., 0.5686, 0.5216, 0.5294], [0.6510, 0.5255, 0.6667,  ..., 0.6000, 0.5961, 0.6196], ..., [0.6235, 0.6392,

Attack Evaluation DL Sample

In [ ]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[12.2449,  2.7617,  2.5413]],  [[12.0375,  3.2163,  2.8674]],  [[12.4965,  3.8211,  3.8029]],  [[12.0638,  3.9614,  3.5344]],  [[12.3805,  3.4368,  3.1378]],  [[11.7753,  2.9400,  2.5359]],  [[12.1240,  3.7199,  2.7683]],  [[11.9614,  3.7038,  3.2117]],  [[11.4893,  2.8110,  2.4000]],  [[12.0184,  3.5900,  3.3133]],  [[12.5584,  3.8429,  3.7609]],  [[12.7498,  4.2989,  3.2611]],  [[12.3147,  3.5242,  2.7728]],  [[12.1801,  3.3081,  3.2056]],  [[12.8172,  3.5859,  2.8155]],  [[11.8787,  2.9492,  2.5386]],  [[12.5542,  3.1082,  3.0152]],  [[12.6553,  3.3358,  2.7283]],  [[11.9960,  2.9726,  2.9327]],  [[11.6972,  3.0410,  2.7115]],  [[12.6478,  3.9242,  3.6788]],  [[12.5895,  2.7901,  2.5623]],  [[11.9965,  3.1878,  3.1599]],  [[12.1037,  2.8593,  2.6725]],  [[11.8756,  3.5196,  3.3826]],  [[12.3221,  3.7395,  3.5548]],  [[12.0674,  3.1243,  3.0504]],  [[12.1973,  3.0358,  2.9905]],  [[12.5676,  3.6918,  3.3290]],  [[11.7444,  2.8265,  2.2521]],  [[12.2648,  3.1581,  2.8681]], 

### Train Attack model

In [ ]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1


### Evaluation

In [ ]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 28.56
